In [3]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "resnet20_quant_4bit"
model = resnet20_quant()

#print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [40, 75]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# This cell won't be given, but students will complete the training

lr = 1e-1
weight_decay = 1e-4
epochs = 80
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [4]:
PATH = "result/"+str(model_name)+"/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9031/10000 (90%)



In [6]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
#print(model.layer1[0])
model.layer1[0].conv1.register_forward_pre_hook(save_output)       ## Input for the module will be grapped
model.layer1[1].conv1.register_forward_pre_hook(save_output)   
        
for i, (input, target) in enumerate(trainloader):
    input, target = input.cuda(), target.cuda() ## transfer to gpu
    output = model(input)
    break

In [7]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha.data.item()   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-1.0000,  2.0000, -1.0000],
          [ 0.0000,  2.0000,  1.0000],
          [-1.0000, -2.0000, -2.0000]],

         [[-3.0000,  0.0000,  3.0000],
          [-3.0000, -0.0000,  2.0000],
          [-2.0000, -2.0000, -1.0000]],

         [[ 1.0000, -0.0000, -2.0000],
          [ 1.0000, -0.0000,  1.0000],
          [ 1.0000,  1.0000,  3.0000]],

         ...,

         [[-3.0000, -3.0000, -5.0000],
          [-1.0000, -2.0000, -3.0000],
          [-3.0000, -1.0000, -3.0000]],

         [[-1.0000, -2.0000,  4.0000],
          [-2.0000, -7.0000, -5.0000],
          [ 3.0000, -7.0000, -4.0000]],

         [[-6.0000, -5.0000, -5.0000],
          [-4.0000, -5.0000, -2.0000],
          [-2.0000, -4.0000, -3.0000]]],


        [[[ 0.0000,  3.0000,  1.0000],
          [-0.0000, -1.0000, -0.0000],
          [-2.0000, -3.0000, -3.0000]],

         [[ 1.0000, -1.0000, -2.0000],
          [-1.0000, -1.0000, -3.0000],
          [ 1.0000,  1.0000, -1.0000]],

         [[-0.0000,  1.0000, -1

In [8]:
x_bit = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv1.act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 6.0000,  5.0000,  5.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 7.0000,  4.0000,  4.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 7.0000,  4.0000,  4.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 7.0000,  4.0000,  4.0000,  ...,  6.0000,  6.0000,  1.0000],
          [ 7.0000,  4.0000,  4.0000,  ...,  4.0000,  4.0000,  1.0000],
          [ 7.0000,  6.0000,  6.0000,  ...,  6.0000,  6.0000,  2.0000]],

         [[ 9.0000,  6.0000,  6.0000,  ...,  1.0000,  1.0000,  0.0000],
          [ 7.0000,  2.0000,  2.0000,  ...,  3.0000,  3.0000,  0.0000],
          [ 7.0000,  2.0000,  2.0000,  ...,  3.0000,  3.0000,  1.0000],
          ...,
          [ 7.0000,  2.0000,  2.0000,  ...,  8.0000,  8.0000,  9.0000],
          [ 7.0000,  2.0000,  2.0000,  ...,  2.0000,  2.0000,  7.0000],
          [ 9.0000,  2.0000,  2.0000,  ...,  2.0000,  2.0000,  2.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  2.0000,  2.0000,  2.0000],
          [ 0.0000,  2.0000,  

In [9]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*x_delta*w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[ -3.6253,   5.8141,   6.2246,  ...,  -2.1205,  -1.3680,   0.6156],
          [-12.1755,  -0.6156,   3.4201,  ...,  -4.1725,  -3.7621,  -3.2149],
          [-12.1755,  -0.6840,   3.2833,  ...,  -4.6513,  -3.7621,  -1.5732],
          ...,
          [-12.1755,  -4.1041, -14.5696,  ..., -25.8558, -26.8819, -26.6767],
          [-12.1755,  -7.1822,  -2.9413,  ...,  -2.3257,  -2.2573,   3.8989],
          [-11.7651,  -7.2506,  -3.9673,  ...,  -0.6156,  -0.9576,   6.2246]],

         [[-11.3547, -12.1071,  -8.9606,  ..., -15.5272, -15.8692, -15.3220],
          [-12.2439, -15.1852, -13.1331,  ..., -15.3220, -16.0744, -14.9116],
          [-12.2439, -15.1168, -13.0647,  ..., -15.8692, -16.2796, -15.5272],
          ...,
          [-12.2439,  -7.1822, -13.5435,  ...,   2.7361,   4.3093,   2.2573],
          [-12.2439,  -7.9346, -16.4848,  ..., -24.1458, -24.2142, -30.3020],
          [-17.2372, -15.2536, -15.5956,  ..., -13.3383, -13.6803, -18.7421]],

         [[-14.6380, -11.9703,

In [10]:
#### input floating number / weight quantized version
conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q

output_ref = conv_ref(x)
print(output_ref)

tensor([[[[-4.5259e+00,  6.2976e+00,  5.5203e+00,  ..., -7.4593e-01,
           -4.5287e-01,  4.6514e-01],
          [-1.2729e+01,  1.0994e+00,  2.5893e+00,  ..., -3.4707e+00,
           -3.2231e+00, -2.4290e+00],
          [-1.2729e+01,  1.4768e+00,  3.5402e+00,  ..., -4.6259e+00,
           -3.4116e+00, -7.9717e-01],
          ...,
          [-1.2729e+01, -2.8148e+00, -1.4435e+01,  ..., -3.1235e+01,
           -3.2716e+01, -3.2594e+01],
          [-1.2729e+01, -6.9835e+00, -2.8568e+00,  ..., -2.4707e+00,
           -1.7177e+00,  6.7158e+00],
          [-1.1311e+01, -6.4798e+00, -3.4295e+00,  ...,  2.5839e-01,
           -1.9357e-01,  8.5417e+00]],

         [[-9.9016e+00, -1.1469e+01, -7.8579e+00,  ..., -1.5759e+01,
           -1.6221e+01, -1.5785e+01],
          [-1.0519e+01, -1.4271e+01, -1.2057e+01,  ..., -1.5780e+01,
           -1.6309e+01, -1.4866e+01],
          [-1.0519e+01, -1.4261e+01, -1.2349e+01,  ..., -1.6472e+01,
           -1.6789e+01, -1.5627e+01],
          ...,
     

In [11]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.1515, device='cuda:0', grad_fn=<MeanBackward0>)


In [12]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std)) ##weight normalisation

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-2.2043e+00,  8.3339e+00,  6.2977e+00,  ..., -1.3743e+00,
           -1.1062e+00, -6.6295e-02],
          [-1.1653e+01,  2.1132e+00,  2.7108e+00,  ..., -4.1844e+00,
           -3.9648e+00, -3.0555e+00],
          [-1.1653e+01,  2.5002e+00,  3.6926e+00,  ..., -5.5649e+00,
           -4.3013e+00, -1.4655e+00],
          ...,
          [-1.1653e+01, -1.7867e+00, -1.3227e+01,  ..., -3.1996e+01,
           -3.2954e+01, -3.3042e+01],
          [-1.1653e+01, -5.9382e+00, -2.1047e+00,  ..., -2.9777e+00,
           -2.2377e+00,  6.2029e+00],
          [-1.0588e+01, -6.0147e+00, -2.6129e+00,  ...,  1.7422e+00,
            1.2087e+00,  9.9199e+00]],

         [[-9.2246e+00, -1.2189e+01, -6.3875e+00,  ..., -1.4734e+01,
           -1.5206e+01, -1.4955e+01],
          [-1.0039e+01, -1.4868e+01, -1.0523e+01,  ..., -1.4555e+01,
           -1.5151e+01, -1.4048e+01],
          [-1.0039e+01, -1.4859e+01, -1.0802e+01,  ..., -1.5299e+01,
           -1.5632e+01, -1.4821e+01],
          ...,
     

In [13]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.7491, device='cuda:0', grad_fn=<MeanBackward0>)


In [14]:
## ResNet Quantization Aware Training for 2-bit precision:: Test results ##
model_name = "resnet20_quant_2bit"
model = resnet20_quant_2bit()
PATH = "result/"+str(model_name)+"/model_best.pth.tar"
checkpoint = torch.load(PATH,map_location=device)
model.load_state_dict(checkpoint['state_dict'])
#device = torch.device("cuda") 

model.to(device)
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7205/10000 (72%)

